# Un'analisi IRT dei quesiti del Bebras italiano

L'*Item Response Theory* (IRT) è una tecnica statistica che si propone di stimare l'*abilità* dei soggetti cui viene sottoposto un test composto da una serie di domande (dette *item*). L'abilità è un tratto *latente*, cioè non osservabile direttamente: l'aver risposto correttamente a molte domande può dipendere sia da elevata abilità, che dalla "facilità" delle domande. La stima si basa quindi su un *modello matematico* (detto *Item Response Function*) della relazione fra abilità e probabilità di rispondere correttamente alla domanda.

Il modello usato in questa analisi è una curva *logistica*, la cui forma è regolata da tre parametri $a, b, c$.
$$ p_i(\theta_j) = c_i + \frac{(1 - c_i)}{1+ e^{-b_i\cdot(\theta_j - a_i)}} $$
La probabilità $p$ di successo in un *item* $i$ è funzione dell'abilità $\theta$ del solutore $j$. I tre parametri modellano le caratteristiche dell'*item*:
- $a$, detto *difficoltà* dell'item: determina il livello di abilità necessaria per avere il 50% di probabilità di successo;
- $b$, detto *differenziazione*: determina quanto influiscono le variazioni di abilità sulla probabilità di successo;
- $c$: fissa una probabilità minima di successo per qualsiasi abilità, dovuta per esempio alla possibilità di scegliere casualmente la risposta giusta ($c$ sarà 0 per un *item* a risposta completamente aperta)

In [ ]:
logistic <- function(theta, a=0, b=1, c=0) { 
    c + (1-c)/(1 + exp(-b*(theta-a)))
}

In [ ]:
library("ggplot2")
data <- data.frame(x=seq(-5,5,.1))
data <- transform(data, 
                  y_mean = logistic(x), 
                  y_hard = logistic(x,a=2), 
                  y_easy=logistic(x,a=-2),
                  y_hdiscrimination=logistic(x,b=2),
                  y_ldiscrimination=logistic(x,b=.5)
                 );

options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_line(aes(y=y_hard), color="red")
p <- p + annotate("text", x=2.1, y=.55, label="a == 2 ~~(plain(difficile))", parse=TRUE, color="red")
p <- p + geom_line(aes(y=y_easy), color="dark green")
p <- p + annotate("text", x=-1.9, y=.55, label="a == -2 ~~(plain(facile))", parse=TRUE, color="dark green")
p <- p + geom_line(aes(y=y_mean), color="blue")
p <- p + annotate("text", x=1.5, y=.8, label="a == 0 ~~(plain(medio))", parse=TRUE, color="blue")
p <- p + geom_line(aes(y=y_hdiscrimination), color="dark cyan")
p <- p + annotate("text", x=1, y=.9, label="a == 0 ~~ b == 2", parse=TRUE, color="dark cyan")
p <- p + geom_line(aes(y=y_ldiscrimination), color="violet")
p <- p + annotate("text", x=1.8, y=.7, label="a == 0 ~~ b == .5", parse=TRUE, color="violet")
p <- p + labs(title=expression(paste("Curve logistiche per diversi valori di ", a, " e ", b, " (",c==0,")")), 
              x="Abilità", y="Probabilità di successo")

ggsave(filename = 'img/logistic.png', plot = p, width = 6, height = 6)
rm(data, p)

![logistic](img/logistic.png)

A ogni *item* o quiz, quindi, viene associata una specifica curva logistica (la sua *item response function*) che può essere dedotta osservando un campione di risolutori cui è stato sottoposto il test. A questo scopo i parametri relativi ai quiz del Bebras italiano 2015 sono stati stimati con un approccio bayesiano, come descritto in [C. Bellettini et al. *"How challenging are Bebras tasks? an IRT analysis based on the performance of Italian students", Proceedings of the 20th annual conference on innovation and technology in computer science education ITiCSE'15* (Vilnius, Lithuania, 2015)](http://dx.doi.org/10.1145/2729094.2742603). I dettagli statistici riguardo la bontà di adattamento del modello sono disponibili [qui](https://mmonga.shinyapps.io/bebrasIT2015).

## I quesiti 2015

In [ ]:
library("rstan"); rstan_options(auto_write = TRUE); options(mc.cores = parallel::detectCores())
load("fit.RData") # carica l'oggetto fit
dfit <- as.data.frame(fit)
obs <- read_rdump("bebras-all.data.R")
source("qnames.R")

cats <- c("kilo", "mega", "giga", "tera", "peta")
categories <- factor(cats, levels=cats, ordered = TRUE)
rm(cats)

`dfit` è il data frame che raccoglie l'esito della simulazione, con 3394 variabili (3168 *ability*, 75 *discrimination*, 75 *difficulty*, 75 *guessing* e la logprobability) e 2000 campioni. `obs` è la lista dei dati effettivamente osservati, cioè 47520 risposte di 3168 squadre a 75 quiz, con 5 categorie. `qnames` contiene i nomi completi dei quiz, come `'2015_Kilo_A1_HU02_Apparecchiare'`.

In [ ]:
obs$rcats <- plyr::aaply(obs$rquiz, 1, function (e){ as.character(categories[ ((e - 1) %/% 15) + 1 ]) })

In [ ]:
library("plyr")
library("dplyr")
teams <- data.frame(id = obs$rtaker, category = obs$rcats) %>% distinct()
get_par_df <- function(df, par, table) {
    df %>% 
    dplyr::select(contains(par)) %>% 
    tidyr::gather(key = "parameter", value, everything()) %>%
    dplyr::mutate("id" = as.numeric(regmatches(parameter, regexpr("[0-9]+",parameter)))) %>%
    dplyr::left_join(table, by = "id") %>%
    transform(category = factor(category, levels=levels(categories), ordered = TRUE))
}

In [ ]:
team_ability <- get_par_df(dfit, "ability", teams)

Il modello è tarato in modo che la media delle abilità di tutte le squadre partecipanti sia 0.

In [ ]:
print(c(summary(team_ability$value), paste("StdDev: ", sd(team_ability$value))), quote = FALSE)

In [ ]:
team_ability.notes <- ddply(team_ability, .(category), summarise, note=sprintf("paste(mu == %s, ' ', sigma == %s)", 
                                                                     format(mean(value), digit = 2), 
                                                                     format(sd(value), digit= 2)))

In [ ]:
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(team_ability, aes(x=value)) 
p <- p + geom_histogram(binwidth=.2) + coord_cartesian(ylim = c(0,.5))
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
p <- p + geom_text(data=team_ability.notes, aes(0, 0.4, label=note), size=3, parse = TRUE)
p <- p + labs(title="Distribuzione dell'abilità dei partecipanti", 
              x="Abilità", y="Densità")
ggsave(filename = 'img/team_ability.png', plot = p, width = 10, height = 2)

![team_ability](img/team_ability.png)

In [ ]:
qorder <- plyr::laply(strsplit(qnames, "_"), function(e){e[3]})
quizzes <- data.frame(id = 1:length(qnames), quiz = qnames, 
                      category = factor(plyr::laply(strsplit(qnames, "_"), function(e){tolower(e[2])}), 
                                        levels=levels(categories), ordered = TRUE),
                      qname = plyr::laply(strsplit(qnames, "_"), function(e){e[5]}),
                      bebras_id = plyr::laply(strsplit(qnames, "_"), function(e){e[4]}),
                      ord = factor(qorder, levels=rev(sort(unique(qorder))), ordered = TRUE),
                      country = plyr::laply(strsplit(qnames, "_"), function(e){substr(e[4],1,2)}),
                      block = plyr::laply(strsplit(qnames, "_"), function(e){substr(e[3],1,1)}))
                      

In [ ]:
quiz_difficulty <- get_par_df(dfit, par = "difficulty", table = quizzes)                                    

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
p <- ggplot(quiz_difficulty, aes(ord, value, fill=category)) 
p <- p + geom_boxplot(outlier.shape = 1, outlier.size = .5)
p <- p + coord_flip() 
p <- p + labs(title="Distribuzione della difficoltà dei quiz", 
              x="Quiz", y="Difficoltà")
ggsave(filename = 'img/quiz_difficulty.png', plot = p, width = 10, height = 10)
rm(p)

![quiz_difficulty](img/quiz_difficulty.png)

In [ ]:
quiz_discrimination <- get_par_df(dfit, par = "discrimination", table = quizzes)

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
p <- ggplot(quiz_discrimination, aes(ord, value, fill=category)) 
p <- p + geom_boxplot(outlier.shape = 1, outlier.size = .5)
p <- p + coord_flip() 
p <- p + labs(title="Distribuzione della differenziazione dei quiz", 
              x="Quiz", y="Differenziazione")
ggsave(filename = 'img/quiz_discrimination.png', plot = p, width = 10, height = 10)
rm(p)

![quiz_discrimination](img/quiz_discrimination.png)

In [ ]:
quiz_guessing <- get_par_df(dfit, par = "guessing", table = quizzes)

##  I singoli quesiti

###  2015_Kilo_A1_HU02_Apparecchiare

![Apparecchiare](screenshot/kilo01-collage.png)

In [ ]:
q <- 'HU02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-HU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-HU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-HU02](img/d-HU02.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-HU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-HU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-HU02](img/a-HU02.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-HU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-HU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-HU02](img/g-HU02.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-HU02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-HU02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-HU02](img/l-HU02.png)

###  2015_Kilo_A2_TR03_Palloncini

![Palloncini](screenshot/kilo02-palloncini.png)

In [ ]:
q <- 'TR03'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-TR03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-TR03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-TR03](img/d-TR03.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-TR03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-TR03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-TR03](img/a-TR03.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-TR03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-TR03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-TR03](img/g-TR03.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-TR03.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-TR03.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-TR03](img/l-TR03.png)

###  2015_Kilo_A3_PL08_Calze

![Calze](screenshot/kilo03-calze.png)

In [ ]:
q <- 'PL08'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-PL08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-PL08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-PL08](img/d-PL08.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-PL08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-PL08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-PL08](img/a-PL08.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-PL08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-PL08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-PL08](img/g-PL08.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-PL08.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-PL08.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-PL08](img/l-PL08.png)

###  2015_Kilo_A4_MY01_Braccialetti

![Braccialetti](screenshot/kilo04-braccialetti.png)

In [ ]:
q <- 'MY01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-MY01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-MY01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-MY01](img/d-MY01.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-MY01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-MY01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-MY01](img/a-MY01.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-MY01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-MY01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-MY01](img/g-MY01.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-MY01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-MY01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-MY01](img/l-MY01.png)

###  2015_Kilo_A5_SK05_Vestiti

![Vestiti](screenshot/kilo05-vestiti.png)

In [ ]:
q <- 'SK05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-SK05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SK05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-SK05](img/d-SK05.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-SK05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SK05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-SK05](img/a-SK05.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-SK05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SK05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-SK05](img/g-SK05.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-SK05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SK05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SK05](img/l-SK05.png)

###  2015_Kilo_B1_LT05_Morra

![Morra](screenshot/kilo06-morra.png)

In [ ]:
q <- 'LT05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-LT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-LT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-LT05](img/d-LT05.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-LT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-LT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-LT05](img/a-LT05.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-LT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-LT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-LT05](img/g-LT05.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-LT05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-LT05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-LT05](img/l-LT05.png)

###  2015_Kilo_B2_JP04_Gru

![Gru](screenshot/kilo07-gru.png)

In [ ]:
q <- 'JP04'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-JP04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-JP04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-JP04](img/d-JP04.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-JP04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-JP04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-JP04](img/a-JP04.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-JP04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-JP04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-JP04](img/g-JP04.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-JP04.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-JP04.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-JP04](img/l-JP04.png)

###  2015_Kilo_B3_TR08_Occhiali

![Occhiali](screenshot/kilo08-occhiali.png)

In [ ]:
q <- 'TR08'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-TR08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-TR08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-TR08](img/d-TR08.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-TR08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-TR08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-TR08](img/a-TR08.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-TR08.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-TR08.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-TR08](img/g-TR08.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-TR08.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-TR08.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-TR08](img/l-TR08.png)

###  2015_Kilo_B4_AT03_Irrigazione

![Irrigazione](screenshot/kilo09-irrigazione.png)

In [ ]:
q <- 'AT03'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-AT03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-AT03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-AT03](img/d-AT03.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-AT03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-AT03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-AT03](img/a-AT03.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-AT03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-AT03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-AT03](img/g-AT03.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-AT03.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-AT03.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-AT03](img/l-AT03.png)

###  2015_Kilo_B5_JP01_Attrezzi

![Attrezzi](screenshot/kilo10-attrezzi.png)

In [ ]:
q <- 'JP01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-JP01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-JP01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-JP01](img/d-JP01.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-JP01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-JP01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-JP01](img/a-JP01.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-JP01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-JP01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-JP01](img/g-JP01.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-JP01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-JP01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-JP01](img/l-JP01.png)

###  2015_Kilo_C1_CZ03_Trasporto

![Trasporto](screenshot/kilo11-trasporto.png)

In [ ]:
q <- 'CZ03'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-CZ03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CZ03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-CZ03](img/d-CZ03.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-CZ03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CZ03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-CZ03](img/a-CZ03.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-CZ03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CZ03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-CZ03](img/g-CZ03.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-CZ03.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CZ03.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CZ03](img/l-CZ03.png)

###  2015_Kilo_C2_SE01_Stelle

![Stelle](screenshot/kilo12-disegnare.png)

In [ ]:
q <- 'SE01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-SE01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SE01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-SE01](img/d-SE01.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-SE01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SE01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-SE01](img/a-SE01.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-SE01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SE01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-SE01](img/g-SE01.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-SE01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SE01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SE01](img/l-SE01.png)

###  2015_Kilo_C3_CZ07_Drone

![Drone](screenshot/kilo13-drone.png)

In [ ]:
q <- 'CZ07'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-CZ07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CZ07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-CZ07](img/d-CZ07.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-CZ07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CZ07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-CZ07](img/a-CZ07.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-CZ07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CZ07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-CZ07](img/g-CZ07.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-CZ07.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CZ07.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CZ07](img/l-CZ07.png)

###  2015_Kilo_C4_CZ01_Noccioline

![Noccioline](screenshot/kilo14-noci.png)

In [ ]:
q <- 'CZ01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-CZ01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CZ01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-CZ01](img/d-CZ01.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-CZ01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CZ01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-CZ01](img/a-CZ01.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-CZ01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CZ01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-CZ01](img/g-CZ01.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-CZ01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CZ01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CZ01](img/l-CZ01.png)

###  2015_Kilo_C5_AT07_Elenco

![Elenco](screenshot/kilo15-elenco.png)

In [ ]:
q <- 'AT07'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-AT07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-AT07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-AT07](img/d-AT07.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-AT07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-AT07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-AT07](img/a-AT07.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-AT07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-AT07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-AT07](img/g-AT07.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-AT07.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-AT07.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-AT07](img/l-AT07.png)

###  2015_Mega_D1_SK02_Tessuto

![Tessuto](screenshot/mega11-tessuto.png)

In [ ]:
q <- 'SK02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-SK02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SK02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-SK02](img/d-SK02.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-SK02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SK02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-SK02](img/a-SK02.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-SK02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SK02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-SK02](img/g-SK02.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-SK02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SK02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SK02](img/l-SK02.png)

###  2015_Mega_D2_IT02_Meteo

![Meteo](screenshot/mega12-meteo.png)

In [ ]:
q <- 'IT02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-IT02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-IT02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-IT02](img/d-IT02.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-IT02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-IT02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-IT02](img/a-IT02.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-IT02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-IT02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-IT02](img/g-IT02.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-IT02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-IT02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-IT02](img/l-IT02.png)

###  2015_Mega_D3_BE04_Scoprirai

![Scoprirai](screenshot/mega13-scoprirai.png)

In [ ]:
q <- 'BE04'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-BE04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-BE04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-BE04](img/d-BE04.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-BE04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-BE04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-BE04](img/a-BE04.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-BE04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-BE04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-BE04](img/g-BE04.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-BE04.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-BE04.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-BE04](img/l-BE04.png)

###  2015_Mega_D4_RU07_Alchimista

![Alchimista](screenshot/mega14-alchimista.png)

In [ ]:
q <- 'RU07'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-RU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-RU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-RU07](img/d-RU07.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-RU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-RU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-RU07](img/a-RU07.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-RU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-RU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-RU07](img/g-RU07.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-RU07.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-RU07.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-RU07](img/l-RU07.png)

###  2015_Mega_D5_HU05_Acrobati

![Acrobati](screenshot/mega15-acrobati.png)

In [ ]:
q <- 'HU05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-HU05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-HU05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-HU05](img/d-HU05.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-HU05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-HU05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-HU05](img/a-HU05.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-HU05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-HU05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-HU05](img/g-HU05.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-HU05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-HU05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-HU05](img/l-HU05.png)

###  2015_Giga_E1_SK04_Luci

![Luci](screenshot/giga11-luci.png)

In [ ]:
q <- 'SK04'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-SK04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SK04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-SK04](img/d-SK04.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-SK04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SK04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-SK04](img/a-SK04.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-SK04.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SK04.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-SK04](img/g-SK04.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-SK04.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SK04.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SK04](img/l-SK04.png)

###  2015_Giga_E2_CH09_Labirinti

![Labirinti](screenshot/giga12-labirinti.png)

In [ ]:
q <- 'CH09'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-CH09.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CH09.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-CH09](img/d-CH09.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-CH09.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CH09.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-CH09](img/a-CH09.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-CH09.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CH09.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-CH09](img/g-CH09.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-CH09.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CH09.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CH09](img/l-CH09.png)

###  2015_Giga_E3_HU07_Pianoforte

![Pianoforte](screenshot/giga13-pianoforte.png)

In [ ]:
q <- 'HU07'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-HU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-HU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-HU07](img/d-HU07.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-HU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-HU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-HU07](img/a-HU07.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-HU07.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-HU07.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-HU07](img/g-HU07.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-HU07.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-HU07.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-HU07](img/l-HU07.png)

###  2015_Giga_E4_CA01_Popolarita

![Popolarita](screenshot/giga14-popolarita.png)

In [ ]:
q <- 'CA01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-CA01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CA01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-CA01](img/d-CA01.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-CA01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CA01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-CA01](img/a-CA01.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-CA01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CA01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-CA01](img/g-CA01.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-CA01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CA01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CA01](img/l-CA01.png)

###  2015_Giga_E5_TW03_LuogoDelitto

![LuogoDelitto](screenshot/giga15-delitto.png)

In [ ]:
q <- 'TW03'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-TW03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-TW03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-TW03](img/d-TW03.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-TW03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-TW03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-TW03](img/a-TW03.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-TW03.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-TW03.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-TW03](img/g-TW03.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-TW03.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-TW03.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-TW03](img/l-TW03.png)

###  2015_Tera_F1_SK01_Robot

![Robot](screenshot/tera11-robot.png)

In [ ]:
q <- 'SK01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-SK01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-SK01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-SK01](img/d-SK01.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-SK01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-SK01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-SK01](img/a-SK01.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-SK01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-SK01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-SK01](img/g-SK01.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-SK01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-SK01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-SK01](img/l-SK01.png)

###  2015_Tera_F2_CA02_Fuochi

![Fuochi](screenshot/tera12-fuochi.png)

In [ ]:
q <- 'CA02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-CA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-CA02](img/d-CA02.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-CA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-CA02](img/a-CA02.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-CA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-CA02](img/g-CA02.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-CA02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CA02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CA02](img/l-CA02.png)

###  2015_Tera_F3_RU02_Ricetta

![Ricetta](screenshot/tera13-ricetta.png)

In [ ]:
q <- 'RU02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-RU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-RU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-RU02](img/d-RU02.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-RU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-RU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-RU02](img/a-RU02.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-RU02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-RU02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-RU02](img/g-RU02.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-RU02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-RU02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-RU02](img/l-RU02.png)

###  2015_Tera_F4_AT01_Famiglia

![Famiglia](screenshot/tera14-famiglia.png)

In [ ]:
q <- 'AT01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-AT01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-AT01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-AT01](img/d-AT01.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-AT01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-AT01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-AT01](img/a-AT01.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-AT01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-AT01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-AT01](img/g-AT01.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-AT01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-AT01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-AT01](img/l-AT01.png)

###  2015_Tera_F5_UA02_Bibite

![Bibite](screenshot/tera15-bibite.png)

In [ ]:
q <- 'UA02'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-UA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-UA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-UA02](img/d-UA02.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-UA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-UA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-UA02](img/a-UA02.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-UA02.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-UA02.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-UA02](img/g-UA02.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-UA02.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-UA02.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-UA02](img/l-UA02.png)

###  2015_Peta_G1_CH01_Calcolatrice

![Calcolatrice](screenshot/peta11-calcolatrice.png)

In [ ]:
q <- 'CH01'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-CH01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CH01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-CH01](img/d-CH01.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-CH01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CH01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-CH01](img/a-CH01.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-CH01.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CH01.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-CH01](img/g-CH01.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-CH01.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CH01.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CH01](img/l-CH01.png)

###  2015_Peta_G2_DE05_Ciondoli

![Ciondoli](screenshot/peta12-ciondoli.png)

In [ ]:
q <- 'DE05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-DE05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-DE05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-DE05](img/d-DE05.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-DE05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-DE05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-DE05](img/a-DE05.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-DE05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-DE05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-DE05](img/g-DE05.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-DE05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-DE05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-DE05](img/l-DE05.png)

###  2015_Peta_G3_IT05_Sassolini

![Sassolini](screenshot/peta13-sassolini.png)

In [ ]:
q <- 'IT05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-IT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-IT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-IT05](img/d-IT05.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-IT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-IT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-IT05](img/a-IT05.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-IT05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-IT05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-IT05](img/g-IT05.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-IT05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-IT05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-IT05](img/l-IT05.png)

###  2015_Peta_G4_JP05_Agenzia

![Agenzia](screenshot/peta14-agenzia.png)

In [ ]:
q <- 'JP05'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-JP05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-JP05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-JP05](img/d-JP05.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-JP05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-JP05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-JP05](img/a-JP05.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-JP05.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-JP05.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-JP05](img/g-JP05.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-JP05.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-JP05.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-JP05](img/l-JP05.png)

###  2015_Peta_G5_CA06_Carte

![Carte](screenshot/peta15-carte.png)

In [ ]:
q <- 'CA06'
d <- quiz_difficulty %>% dplyr::filter(bebras_id == q) %>% select(value, category)
a <- quiz_discrimination %>% dplyr::filter(bebras_id == q) %>% select(value, category)
g <- quiz_guessing %>% dplyr::filter(bebras_id == q) %>% select(value, category)

#### Difficoltà

In [ ]:
if (!file.exists('img/d-CA06.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(d, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(d$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/d-CA06.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(d$value)

![d-CA06](img/d-CA06.png)

#### Differenziazione

In [ ]:
if (!file.exists('img/a-CA06.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(a, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(a$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/a-CA06.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(a$value)

![a-CA06](img/a-CA06.png)

#### Guessing

In [ ]:
if (!file.exists('img/g-CA06.png')) {
options(repr.plot.width=10, repr.plot.height=2)
p <- ggplot(g, aes(x=value))
p <- p + geom_histogram(binwidth=0.1*sd(g$value)) 
p <- p + aes(y=..density..)
p <- p + facet_grid(. ~ category)
ggsave(filename = 'img/g-CA06.png', plot = p, width = 10, height = 2)
rm(p)
}
summary(g$value)

![g-CA06](img/g-CA06.png)

#### Curva Item-Response

In [ ]:
if (!file.exists('img/l-CA06.png')) {
data <- data.frame(x=seq(-5,5,.1))
options(repr.plot.width=6, repr.plot.height=6)
p <- ggplot(data, aes(x=x))
p <- p + scale_x_continuous()
p <- p + geom_hline(yintercept=0.5, linetype="dashed")
p <- p + geom_vline(xintercept=0.0, linetype="dashed")
dall <- data.frame()
for (i in 1:length(d$value)) {
 ldata <- bind_cols(data,  data.frame(a=rep(d$value[i], times=length(data$x)),
                                      b=rep(a$value[i], times=length(data$x)),
                                      c=rep(g$value[i], times=length(data$x)),
                                      category=rep(d$category[i], times=length(data$x))
                                   ))
 ldata <- mutate(ldata, y = logistic(x, a, b, c))
  dall <- bind_rows(dall, ldata)
}
 p <- p + geom_point(data=dall, aes(x=x, y=y)) + geom_smooth(data=dall, aes(x=x, y=y))
 p <- p + facet_grid(. ~ category)
 ggsave(filename = 'img/l-CA06.png', plot = p, width = 6, height = 6)
 rm(data, dall, ldata, p)
}

![l-CA06](img/l-CA06.png)